In [82]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import pickle
from datetime import datetime

In [83]:
%%time
test_df = pd.read_csv("./data/output6.csv")

CPU times: user 437 ms, sys: 140 ms, total: 577 ms
Wall time: 624 ms


In [84]:
test_df

,index,TIMESTAMP,TIMESTAMP_BK
0,3220254,1579888192.09,1579685016.00
1,3220255,1579685127.33,0.00
2,3220256,1579685226.78,1579685246.00
3,3220257,1579685337.48,0.00
4,3220258,1579685483.07,0.00
...,...,...,...
1557459,4777713,1579883543.64,1579883967.00
1557460,4777714,1579884096.41,1579886880.00
1557461,4777715,1579886997.28,0.00
1557462,4777716,1579887145.04,0.00


In [85]:
%%time
gap_list = []
for idx, row in test_df.iterrows():
    timestamp = row['TIMESTAMP_BK']
    if (timestamp == 0):
        prev_timestamp = test_df.iloc[idx-1]['TIMESTAMP_BK']
        if (prev_timestamp != 0):
            gap_start = idx-1
            pass
        next_timestamp = test_df.iloc[idx+1]['TIMESTAMP_BK']
        if (next_timestamp!=0):
            gap_end = idx+1
            gap_list.append((gap_start, gap_end))
            
            #cnt = cnt + 1
            #if cnt>5:
            #    break
            
len(gap_list)

CPU times: user 10min 26s, sys: 5.76 s, total: 10min 32s
Wall time: 11min 33s


250529

In [86]:
gap_list[:5]

[(0, 2), (2, 5), (5, 8), (8, 13), (13, 18)]

In [87]:
%%time
pd.options.display.float_format = "{:.2f}".format
# cnt = 0

for gap in gap_list:
    gap_start     = gap[0]
    gap_end       = gap[1]
    scale_min     = test_df.iloc[gap_start]['TIMESTAMP_BK']
    scale_max     = test_df.iloc[gap_end  ]['TIMESTAMP_BK']
    bk_timestamps = test_df.iloc[gap_start:gap_end+1]['TIMESTAMP'].values
    
    bk_timestamps[0] = scale_min
    
    rescaled = np.interp(
        bk_timestamps, 
        (bk_timestamps.min(), bk_timestamps.max()), 
        (scale_min, scale_max)
    )
    iter_count = gap_end - gap_start
    for i in range(0, iter_count+1):
        idx = gap_start+i
        test_df.at[idx, 'TIMESTAMP_SCALED'] = rescaled[i]
        test_df.at[idx, 'DATETIME_SCALED' ] = datetime.utcfromtimestamp(rescaled[i])
        
#     cnt = cnt+1
#     if cnt>1:
#        break

CPU times: user 4min 50s, sys: 3.1 s, total: 4min 53s
Wall time: 5min 20s


In [88]:
test_df.head(60)

,index,TIMESTAMP,TIMESTAMP_BK,TIMESTAMP_SCALED,DATETIME_SCALED
0,3220254,1579685016.00,1579685016.00,1579685016.00,2020-01-22 09:23:36.000000
1,3220255,1579685127.33,0.00,1579685137.48,2020-01-22 09:25:37.481148
2,3220256,1579685246.00,1579685246.00,1579685246.00,2020-01-22 09:27:26.000000
3,3220257,1579685337.48,0.00,1579685333.59,2020-01-22 09:28:53.591269
4,3220258,1579685483.07,0.00,1579685473.01,2020-01-22 09:31:13.008599
5,3220259,1579685572.00,1579685572.00,1579685572.00,2020-01-22 09:32:52.000000
6,3220260,1579685803.09,0.00,1579685777.78,2020-01-22 09:36:17.779127
7,3220261,1579685942.90,0.00,1579685902.27,2020-01-22 09:38:22.273511
8,3220262,1579686153.00,1579686153.00,1579686153.00,2020-01-22 09:42:33.000000
9,3220263,1579686249.65,0.00,1579686283.44,2020-01-22 09:44:43.438368


In [89]:
%%time
for idx, row in test_df.iterrows():
    timestamp_scaled = row['TIMESTAMP_SCALED']
    if (pd.isna(timestamp_scaled)):
        test_df.at[idx, 'TIMESTAMP_SCALED'] = row['TIMESTAMP_BK']

CPU times: user 3min 24s, sys: 2.22 s, total: 3min 26s
Wall time: 3min 46s


In [90]:
test_df_bk = test_df
test_df = test_df_bk

In [91]:
test_df.head(60)

,index,TIMESTAMP,TIMESTAMP_BK,TIMESTAMP_SCALED,DATETIME_SCALED
0,3220254,1579685016.00,1579685016.00,1579685016.00,2020-01-22 09:23:36.000000
1,3220255,1579685127.33,0.00,1579685137.48,2020-01-22 09:25:37.481148
2,3220256,1579685246.00,1579685246.00,1579685246.00,2020-01-22 09:27:26.000000
3,3220257,1579685337.48,0.00,1579685333.59,2020-01-22 09:28:53.591269
4,3220258,1579685483.07,0.00,1579685473.01,2020-01-22 09:31:13.008599
5,3220259,1579685572.00,1579685572.00,1579685572.00,2020-01-22 09:32:52.000000
6,3220260,1579685803.09,0.00,1579685777.78,2020-01-22 09:36:17.779127
7,3220261,1579685942.90,0.00,1579685902.27,2020-01-22 09:38:22.273511
8,3220262,1579686153.00,1579686153.00,1579686153.00,2020-01-22 09:42:33.000000
9,3220263,1579686249.65,0.00,1579686283.44,2020-01-22 09:44:43.438368


In [92]:
test_df = test_df[['index', 'TIMESTAMP_SCALED', 'TIMESTAMP_BK']].rename(columns={"TIMESTAMP_SCALED": "TIMESTAMP"})

In [93]:
test_df.head(50)

,index,TIMESTAMP,TIMESTAMP_BK
0,3220254,1579685016.00,1579685016.00
1,3220255,1579685137.48,0.00
2,3220256,1579685246.00,1579685246.00
3,3220257,1579685333.59,0.00
4,3220258,1579685473.01,0.00
5,3220259,1579685572.00,1579685572.00
6,3220260,1579685777.78,0.00
7,3220261,1579685902.27,0.00
8,3220262,1579686153.00,1579686153.00
9,3220263,1579686283.44,0.00


In [94]:
# test_df.to_csv("./out/test_df_timestamp_adjusted.csv")
test_df[['index', 'TIMESTAMP']].to_csv("./out/output_ready_6.csv", index=False)

In [49]:
# test_2 = pd.read_csv("./out/output_ready_best.csv")

In [51]:
# pd.merge(test_2, test_df, on='index', how='inner')

,index,TIMESTAMP_x,TIMESTAMP_y,TIMESTAMP_BK
0,3220254,1579685016.00,1579685016.00,1579685016.00
1,3220255,1579685137.05,1579685131.00,0.00
2,3220256,1579685246.00,1579685246.00,1579685246.00
3,3220257,1579685339.27,1579685341.34,0.00
4,3220258,1579685475.31,1579685475.20,0.00
...,...,...,...,...
1557459,4777713,1579883967.00,1579883967.00,1579883967.00
1557460,4777714,1579886880.00,1579886880.00,1579886880.00
1557461,4777715,1579887223.60,1579887932.62,0.00
1557462,4777716,1579887676.35,1579887983.28,0.00
